<a href="https://colab.research.google.com/github/ValentinaZubareva2906/deep_learning_mipt/blob/main/chain/3_1_itog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-classic openai langchain-openai langchain-community -U -q

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain_classic import PromptTemplate
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
from langchain_classic.schema.output_parser import StrOutputParser

In [ ]:
from langchain_openai import ChatOpenAI
from getpass import getpass

course_api_key= "sk-CTWDfcT-MqN2gUhZh_3qbA"

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [ ]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [ ]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]
    text = re.sub(r'[¿¡£]', '', text)
    cleaned = re.sub(r"\s+", " ", text).strip()

    return {"text": cleaned}

Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках. НЕ ОБЯЗАТЕЛЬНЫЙ ШАГ

In [ ]:
# Определим схемы ответа
language_schema = ResponseSchema(
    name="language",
    description=("Извлеки язык, на котором был написан данный текст."
        "Пример: 'Русский' → 'Russian'."
        "Пример: 'Deutsch' → 'German'."
    )
)


main_character_schema = ResponseSchema(
    name="main_character",
    description=("Извлеки имя главного героя из приведенного текста."
        "Имя героя должно быть на том языке, на котором написан {text}"
    )
)

response_schemas = [language_schema, main_character_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [ ]:
template = """Из следующего текста извлеки информацию:

Текст: {text}

{format_instructions}
"""

In [ ]:
prompt = PromptTemplate(input_variables=['text'],
                        template=template,
                        partial_variables={"format_instructions": format_instructions})

Создадим цепочку с помощью `LCEL`

In [ ]:
chain = clean_text | prompt | llm | output_parser

In [ ]:
results = []

for raw_text in tqdm(df["raw_text"]):
    try:
        answer = chain.invoke({"text": raw_text})
        cleaned = clean_text({"text": raw_text})
        result = {
            "text": cleaned["text"],
            **answer
        }
        results.append(result)
    except Exception as e:
        print(f"Ошибка для текста '{raw_text}': {e}")
        results.append({
            "text": cleaned["text"],
            "language": None,
            "main_character": None
        })

 15%|█▌        | 2/13 [00:02<00:15,  1.39s/it]

Ошибка для текста 'Le soleil se couchait, jetant de longues ombres sur la petite ville. Pierre, un homme d'âge moyen avec un cœur plein de rêves, était assis sur le porche de sa vieille maison. Ses amis, Jacques et Marie, étaient là avec lui, partageant des histoires de leur jeunesse. Ils étaient tous de la même ville, et leurs vies étaient entrelacées de manière qu'ils n'auraient jamais pu imaginer. Jacques, un homme robuste à la voix tonitruante, était le forgeron de la ville. Il était un homme de peu de mots, mais ses actions parlaient beaucoup. Marie, en revanche, était l'institutrice de la ville. Elle était une femme de grande sagesse et de gentillesse, toujours prête à tendre la main. Alors qu'ils étaient assis là, se remémorant le passé, un sentiment de nostalgie les envahit. Ils se souvenaient des jours où ils étaient jeunes et pleins d'espoir, quand le monde semblait plein de possibilités. Mais maintenant, en regardant le soleil couchant, ils se rendaient compte que ces jours 

 62%|██████▏   | 8/13 [00:10<00:05,  1.15s/it]

Ошибка для текста 'В тихом уголке старого города, где узкие уло¿чки переплетались в бесконечный лабиринт, стоял небольшой домик. В нем жила Анн¡а. Она была необычной девушкой, с добрыми глазами и теплой улыбкой. В ее жизни было много интересных персонажей: старый сосед Петр, который всегда рассказывал ей истории о прошлом города, ма¡ленькая девочка М¡аша, которая любила играть с Ан¡ной в саду, и ее лучший друг Иван, который всегда был рядом в трудные моменты. Вместе они переживали множество приключений, и каждый день был для них новым открытием.': Error code: 400 - {'error': {'message': 'litellm.BadRequestError: OpenAIException - The OpenAI account associated with this API key has been deactivated. If you are the developer for this OpenAI app, please check your email for more information. If you are seeing this error while using another app or site, please reach out to them for more help.. Received Model Group=gpt-4o-mini\nAvailable Model Group Fallbacks=None', 'type': 'invalid_request

 85%|████████▍ | 11/13 [00:14<00:02,  1.29s/it]

Ошибка для текста 'Dans un petit village niché entre les montagnes et la mer, vivait une fi¿lle nommée Sophie. Elle était connue pour sa gentillesse et son amour pour la nature. Sa vie était remplie de personnages intéressants : le vieux pêcheur Jean, qui lui racontait des histoires de la mer, le jeune garçon Thomas, qui la rejoignait dans ses aventures, et sa meilleure amie Emilie, qui était toujours là pour elle. Ensemble, ils ont vécu de nombreuses aventures, et chaque jour était une nouvelle découverte pour eux.': Error code: 400 - {'error': {'message': 'litellm.BadRequestError: OpenAIException - The OpenAI account associated with this API key has been deactivated. If you are the developer for this OpenAI app, please check your email for more information. If you are seeing this error while using another app or site, please reach out to them for more help.. Received Model Group=gpt-4o-mini\nAvailable Model Group Fallbacks=None', 'type': 'invalid_request_error', 'param': None, 'code'

100%|██████████| 13/13 [00:18<00:00,  1.40s/it]


In [ ]:
new_df = pd.DataFrame(results)
new_df.head(15)

,text,language,main_character
0,"The sun was setting, casting long shadows over...",English,John
1,"The sun was setting, casting long shadows over...",None,None
2,"El sol se estaba poniendo, proyectando largas ...",Spanish,Carlos
3,"La ciudad estaba llena de vida, sus calles lle...",Spanish,Juan
4,"La ville était pleine de vie, ses rues remplie...",French,Jean
5,"Die Stadt war voller Leben, ihre Straßen gefül...",German,Johann
6,Die Sonne ging unter und warf lange Schatten ü...,German,Hans
7,Die Sonne ging unter und warf lange Schatten ü...,None,None
8,In a small town nestled between the mountains ...,English,Laura
9,En un pequeño pueblo situado entre las montaña...,Spanish,Maria


In [ ]:
new_df[['text', 'language', 'main_character']].to_csv('3.2.9solution.csv', index=False)